# Testing the Custom MEES SEG-Y Reader

The Python code below was received from the researcher who collected and disseminates the MEES data (http://www.mseel.org/research/).

In [1]:
import numpy as np
from numpy import mean, sqrt, square, arange
import os
import re
import glob
from struct import unpack

In [2]:
def ibm2ieee(ibm):
    if ibm == 0:
        return 0.0
    sign = ibm >> 31 & 0x01
    exponent = ibm >> 24 & 0x7f
    mantissa = (ibm & 0x00ffffff) / float(pow(2, 24))
    return (1 - 2 * sign) * mantissa * pow(16, exponent - 64)

In [3]:
segy_file = '/Users/ajelenak/Downloads/NNE_MIP3H_v00001_NNE_MW3H_TW3H_S02F_20151107_052552_3_SegyPhase.sgy'

In [4]:
with open(segy_file, 'rb') as f:
    segy_data = f.read()

In [5]:
len(segy_data)

118441921

In [6]:
text_hdr=segy_data[:3200].decode('utf-8')
text_hdr

'C 1 CLIENT North East Natural NNE SCHLUMBERGER CREW NO Pittsburg................C 2 WELL MIP3H LOCATION Morgantown West Virginia................................C 3 1 November 2015 OBSERVER Dowdell/Miller.....................................C 4 INSTRUMENT: SLB HDVS   SERIAL NO ENP26505...................................C 5 493 TRACES/RECORD   0 AUXILIARY TRACES/RECORD...............................C 6 SAMPLE INTERVAL 500 SAMPLES/TRACE  60000....................................C 7 RECORDING FORMAT 1..........................................................C 8.............................................................................C 9 GAIN  TYPE: 0...............................................................C10 FILTERS: ALIAS 0 HZ, NOTCH 0 HZ, BAND 0 HZ, SLOPE 0 DB/OCT..................C11 SOURCE: 0...................................................................C12 PATTERN:   LENGTH       WIDTH...............................................C13 SWEEP: START 0 HZ END 0 HZ  LENGTH 

In [7]:
ijnum = int.from_bytes(segy_data[3200:3204],byteorder='big')
ireel = int.from_bytes(segy_data[3208:3212],byteorder='big')
itrac = int.from_bytes(segy_data[3212:3214],byteorder='big')
isamp = int.from_bytes(segy_data[3216:3218],byteorder='big')
nsamp = int.from_bytes(segy_data[3220:3222],byteorder='big')
icode = int.from_bytes(segy_data[3224:3226],byteorder='big')
imeas = int.from_bytes(segy_data[3254:3256],byteorder='big')
iform = int.from_bytes(segy_data[3500:3502],byteorder='big')/256
iextd = int.from_bytes(segy_data[3504:3505],byteorder='big')
insamp = int.from_bytes(segy_data[3506:3510], byteorder='big')

In [8]:
ijnum

1

In [9]:
ireel

1

In [10]:
itrac

493

In [11]:
isamp

500

In [12]:
nsamp

60000

In [13]:
icode

1

In [14]:
imeas

2

In [15]:
iform

1.0

In [16]:
iextd

0

In [17]:
insamp

60000

In [23]:
trace_data=[]
trace_depth = np.zeros(shape=(itrac))
data_array  = np.zeros(shape=(insamp+1,itrac))
for trace in range(1,itrac):
    ibyte_start = 3840+(trace-1)*240+(trace-1)*insamp*4
    trace_hdr = segy_data[ibyte_start-240:ibyte_start]
    irec  = int.from_bytes(trace_hdr[8:12],byteorder='big')
    itnum = int.from_bytes(trace_hdr[12:16],byteorder='big')
    idpth = int.from_bytes(trace_hdr[40:44],byteorder='big')
    iscal = int.from_bytes(trace_hdr[70:72],byteorder='big')
    iyear = int.from_bytes(trace_hdr[156:158],byteorder='big')
    iday  = int.from_bytes(trace_hdr[158:160],byteorder='big')
    ihour = int.from_bytes(trace_hdr[160:162],byteorder='big')
    imin  = int.from_bytes(trace_hdr[162:164],byteorder='big')
    isec  = int.from_bytes(trace_hdr[164:166],byteorder='big')
    icodt = int.from_bytes(trace_hdr[166:168],byteorder='big')
    insamp= int.from_bytes(trace_hdr[224:228],byteorder='big')
    iusec = int.from_bytes(trace_hdr[232:236],byteorder='big')
    ## print(trace,idpth,iyear,iday,ihour,imin,isec,iusec,insamp)
    trace_depth[trace] =(idpth/iscal)
    for idata in range (ibyte_start, ibyte_start+(insamp*4),4):
        ibm=int.from_bytes(segy_data[idata:idata+4],byteorder='big')
        trace_data.append(ibm2ieee(ibm))
    data_array[1:insamp,trace]=np.array(trace_data[1:insamp])
    trace_data = []

In [26]:
type(data_array)

numpy.ndarray

In [27]:
data_array.shape

(60001, 493)

In [29]:
data_array[0,:].

0.0

In [30]:
np.median(data_array, axis=0)

array([ 0.00000000e+00, -1.75561581e-04, -1.06257663e-04,  1.23649617e-04,
       -1.81966374e-04,  1.88086124e-05, -8.01474089e-05, -7.93197833e-05,
        2.75797676e-04, -6.49049355e-07,  1.61770979e-04, -5.24629286e-05,
        4.51734959e-05,  2.32064776e-04, -1.37439070e-04, -3.22235748e-04,
        5.37919259e-05,  2.75401631e-04,  6.01294450e-05,  1.18622804e-04,
       -1.25252154e-06, -2.08995480e-04,  2.82609370e-04, -3.67563043e-05,
       -6.90268789e-05, -1.03080121e-04, -4.63209348e-04, -1.41670651e-04,
        5.70092350e-04,  8.06291791e-05,  1.55414018e-05, -1.19148957e-04,
        2.04071766e-04,  9.30770329e-06, -4.19739517e-05,  1.23676931e-04,
        6.94903804e-07, -6.76773197e-05,  5.18362212e-05,  7.47603190e-05,
        1.53624715e-04,  1.85516546e-05, -2.99426960e-04, -1.62274635e-04,
       -5.88976400e-05, -1.06105028e-04,  1.41542609e-04, -2.59330318e-06,
       -3.24602297e-05, -4.59873991e-06, -2.28521414e-04,  2.47437973e-04,
       -2.21106893e-05,  